## Import Library

In [1]:
import requests 
import tensorflow as tf
import numpy as np
import json
import pandas as pd
import base64
from sklearn.model_selection import train_test_split

## Data Load

In [2]:
df = pd.read_csv('dataset/heart_attack_dataset.csv')

if 'Patient ID' in df.columns:
    df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere', 'Diet'], inplace=True)

df.head()

,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Previous Heart Problems,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Heart Attack Risk
0,67,Male,208,72,0,0,1,0,0,4.168189,0,0,9,6.615001,261404,31.251233,286,0,6,0
1,21,Male,389,98,1,1,1,1,1,1.813242,1,0,1,4.963459,285768,27.194973,235,1,7,0
2,21,Female,324,72,1,0,0,0,0,2.078353,1,1,9,9.463426,235282,28.176571,587,4,4,0
3,84,Male,383,73,1,1,1,0,1,9.828130,1,0,9,7.648981,125640,36.464704,378,3,4,0
4,66,Male,318,93,1,1,1,1,0,5.804299,1,0,6,1.514821,160555,21.809144,231,1,5,0


In [3]:
X = df.drop(columns=['Heart Attack Risk'])
y = df['Heart Attack Risk']

In [4]:
row = np.array(X.iloc[0].values)
label = y.iloc[0]

print('Test row: ', row, '\n')
print('Test label: ', label)

Test row:  [67 'Male' 208 72 0 0 1 0 0 4.168188835442079 0 0 9 6.61500145291406
 261404 31.2512327252954 286 0 6] 

Test label:  0


In [5]:
print(X.iloc[0])

Age                                       67
Sex                                     Male
Cholesterol                              208
Heart Rate                                72
Diabetes                                   0
Family History                             0
Smoking                                    1
Obesity                                    0
Alcohol Consumption                        0
Exercise Hours Per Week             4.168189
Previous Heart Problems                    0
Medication Use                             0
Stress Level                               9
Sedentary Hours Per Day             6.615001
Income                                261404
BMI                                31.251233
Triglycerides                            286
Physical Activity Days Per Week            0
Sleep Hours Per Day                        6
Name: 0, dtype: object


In [6]:
columns_to_exclude = ['Exercise Hours Per Week', 'Sedentary Hours Per Day', 'BMI']

converted_row = []
for i, value in enumerate(row):
    column_name = X.columns[i]
    if column_name in columns_to_exclude:
        converted_row.append(value)  # Keep the original value
    elif column_name == 'Sex':
        converted_row.append(str(value))  # Convert Sex to string
    else:
        converted_row.append(int(value))  # Convert to int

In [7]:
def create_tf_example(row):
    feature = {
        'Age': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[0]])),
        'Sex': tf.train.Feature(bytes_list=tf.train.BytesList(value=[row[1].encode('utf-8')])),
        'Cholesterol': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[2]])),
        'Heart Rate': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[3]])),
        'Diabetes': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[4]])),
        'Family History': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[5]])),
        'Smoking': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[6]])),
        'Obesity': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[7]])),
        'Alcohol Consumption': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[8]])),
        'Exercise Hours Per Week': tf.train.Feature(float_list=tf.train.FloatList(value=[row[9]])),
        'Previous Heart Problems': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[10]])),
        'Medication Use': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[11]])),
        'Stress Level': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[12]])),
        'Sedentary Hours Per Day': tf.train.Feature(float_list=tf.train.FloatList(value=[row[13]])),
        'Income': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[14]])),
        'BMI': tf.train.Feature(float_list=tf.train.FloatList(value=[row[15]])),
        'Triglycerides': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[16]])),
        'Physical Activity Days Per Week': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[17]])),
        'Sleep Hours Per Day': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[18]]))
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()

In [8]:
serialized_example = create_tf_example(converted_row)

## Make Prediction Request

In [10]:
json_data = json.dumps(
    {
        "instances": [{"examples": {"b64": base64.b64encode(serialized_example).decode('utf-8')}}]
    }
)

endpoint = "http://localhost:8080/v1/models/heart-attack-risk-model:predict"

response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})
    
prediction = np.argmax(response.json()["predictions"][0])

if prediction == 1:
    print("The person has heart attack risk.")
else:
    print("The person does not have heart attack risk.")

The person does not have heart attack risk.
